In [2]:
import pandas as pd
import numpy as np
import rpy2.robjects as robjects
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()

from PyCRO_SL import *
from PyCRO_SL.CRO_SL import CRO_SL
from PyCRO_SL.AbsObjetiveFunc import AbsObjetiveFunc
from PyCRO_SL.SubstrateInt import SubstrateInt
from PyCRO_SL.SubstrateReal import SubstrateReal
import random

In [3]:
class HydroProblemSemidist(AbsObjetiveFunc):
    def __init__(self, rscript_name='exec_problem_semidist.R', metric="MSE", model_used=0, basin_code=-1, prev_q=0):         
        # Defining the R script and loading the instance in Python
        r = robjects.r
        r['source'](rscript_name)

        inf_lim = np.array([1e-4,10,0,1e-4,1e-4,2,-6])
        sup_lim = np.array([1,2000,1,100,1,11,1])
        self.size = 7

        self.metric = metric
        self.model_used = model_used
        self.basin_code = basin_code
        self.prev_q = prev_q

        opt = "min"
        if metric in ["KGE", "NSE"]:
            opt = "max"

        # Loading the function we have defined in R.
        self.exec_function_r = robjects.globalenv['eval_basin_param']
        super().__init__(self.size, opt, sup_lim, inf_lim)

    def objetive(self, solution):
        metrics = self.exec_function_r(self.model_used, solution, self.basin_code, self.prev_q)
        if self.metric == "MSE":
            return float(metrics[0])
        elif self.metric == "RMSE":
            return float(metrics[1])
        elif self.metric == "NSE":
            return float(metrics[3])
        elif self.metric == "R2":
            return abs(1 - float(metrics[4]))
        elif self.metric == "KGE":
            return float(metrics[5])
        

    def random_solution(self):
        return (self.sup_lim-self.inf_lim) * np.random.random(self.size) + self.inf_lim
    
    def check_bounds(self, solution):
        return solution.clip(self.inf_lim, self.sup_lim)
    

In [25]:
substrates_real = [
    SubstrateReal("Gauss", {"F": 0.0001}),
    SubstrateReal("Cauchy", {"F": 0.01}),
    SubstrateReal("DE/best/2", {"F": 0.7, "Cr":0.7}),
    SubstrateReal("BLXalpha", {"Cr": 0.35}),
    # SubstrateReal("Firefly", {"a": 0.7, "b": 1, "d": 0.95, "g": 10}),
    # SubstrateReal("Perm", {"Cr": 4/7}),
]

params = {
    "popSize": 125,
    "rho": 0.6,
    "Fb": 0.98,
    "Fd": 0.2,
    "Pd": 0.99,
    "k": 3,
    "K": 10,
    "group_subs": False,

    "stop_cond": "ngen",
    "time_limit": 400.0,
    #"Ngen": 1,
    "Ngen": 100,
    "Neval": 3e4,
    "fit_target": 1000,

    "verbose": True,
    "v_timer": 1,

    "dynamic": True,
    "dyn_method": "fitness",
    "dyn_metric": "best",
    "dyn_steps": 500,
    "prob_amp": 0.02
}

In [26]:

model = 2
target = "NSE"

rscript_name = "exec_optim_semidist.R"

r = robjects.r
r['source'](rscript_name)
get_basin_q = robjects.globalenv['get_basin_q']

basins = pd.read_csv("basins.txt")
#basins.sort_values(by=["order"])
agg_q = {}
basin_params = {}

prev_q = 0
for idx in basins.index:
    basin_code = basins["code"][idx]
    codedown = basins["codedown"][idx]

    prev_q = 0
    if basin_code in agg_q:
        prev_q = agg_q[basin_code]        

    objfunc = HydroProblemSemidist(rscript_name, target, model, basin_code, prev_q)
    c = CRO_SL(objfunc, substrates_real, params)
    
    print(f"Optimizing basin with code {basin_code}.\n\n")

    c.safe_optimize()
    basin_params[basin_code] = c.best_solution()

    if codedown not in agg_q:
        agg_q[codedown] = 0

    agg_q[codedown] += get_basin_q(model, c.best_solution()[0], basin_code, prev_q)


Rows: 4 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): code, order, codedown, supha

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 4 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): code, order, codedown, supha

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Optimizing basin with code 3001.


Time Spent 254.59s:
	Generation: 1
	Best fitness: 0.486397
	Evaluations of fitness: 241
	Substrate probability:
		Gauss:    0.25
		Cauchy:   0.25
		DE/best/2:0.25
		BLXalpha: 0.25

Time Spent 360.43s:
	Generation: 2
	Best fitness: 0.535832
	Evaluations of fitness: 341
	Substrate probability:
		Gauss:    0.144851
		Cauchy:   0.10304
		DE/best/2:0.6

R[write to console]: 



RRuntimeError: Cannot determine terminal size (system error 25, Inappropriate ioctl for device) @tty.c:31 (clic_tty_size)

In [15]:
result = []
for i in basins["code"]:
    result += [basin_params[i][0]]
result = np.array(result)

np.savetxt("config_nooptim.csv", result, delimiter=",")